This notebook shows usage of SageMaker Managed Spot infrastructure for XGBoost training. Below we show how Spot instances can be used for the ‘algorithm mode’ and ‘script mode’ training methods with the XGBoost container.

Managed Spot Training uses Amazon EC2 Spot instance to run training jobs instead of on-demand instances. You can specify which training jobs use spot instances and a stopping condition that specifies how long Amazon SageMaker waits for a job to run using Amazon EC2 Spot instances.

This notebook was tested in Amazon SageMaker Studio on a ml.t3.medium instance with Python 3 (Data Science) kernel.

In this notebook we will perform XGBoost training as described `here <>`__. See the original notebook for more details on the data.

### Setup variables and define functions

In [1]:
%%time

import io
import os
import boto3
import sagemaker

role = sagemaker.get_execution_role()
region = boto3.Session().region_name

# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket here if you wish.
bucket = sagemaker.Session().default_bucket()
prefix = "sagemaker/DEMO-xgboost-spot"
# customize to your bucket where you have would like to store the data

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/base_serializers.py:28: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.22.4)
  import scipy.sparse


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
CPU times: user 1.77 s, sys: 148 ms, total: 1.92 s
Wall time: 2.94 s


In [2]:
print(bucket)
print(prefix)

sagemaker-us-east-1-381492029181
sagemaker/DEMO-xgboost-spot


### Fetching the dataset

In [3]:
%%time
s3 = boto3.client("s3")
# Load the dataset
FILE_DATA = "abalone"
s3.download_file(
    f"sagemaker-example-files-prod-{region}",
    f"datasets/tabular/uci_abalone/abalone.libsvm",
    FILE_DATA,
)
sagemaker.Session().upload_data(FILE_DATA, bucket=bucket, key_prefix=prefix + "/train")
sagemaker.Session().upload_data(FILE_DATA, bucket=bucket, key_prefix=prefix + "/validation")

CPU times: user 396 ms, sys: 23.4 ms, total: 419 ms
Wall time: 906 ms


's3://sagemaker-us-east-1-381492029181/sagemaker/DEMO-xgboost-spot/validation/abalone'

### Obtaining the latest XGBoost container

We obtain the new container by specifying the framework version (1.7-1). This version specifies the upstream XGBoost framework version (1.7) and an additional SageMaker version (1). If you have an existing XGBoost workflow based on the previous (1.0-1, 1.2-2, 1.3-1 or 1.5-1) container, this would be the only change necessary to get the same workflow working with the new container.

In [4]:
container = sagemaker.image_uris.retrieve("xgboost", region, "1.7-1")

### Training the XGBoost model

After setting training parameters, we kick off training, and poll for status until training is completed, which in this example, takes few minutes.

To run our training script on SageMaker, we construct a sagemaker.xgboost.estimator.XGBoost estimator, which accepts several constructor arguments:

* entry_point: The path to the Python script SageMaker runs for training and prediction.
* role: Role ARN
* hyperparameters: A dictionary passed to the train function as hyperparameters.
* train_instance_type (optional): The type of SageMaker instances for training. Note: This particular mode does not currently support training on GPU instance types.
* sagemaker_session (optional): The session used to train on Sagemaker.

In [5]:
hyperparameters = {
    "max_depth": "5",
    "eta": "0.2",
    "gamma": "4",
    "min_child_weight": "6",
    "subsample": "0.7",
    "objective": "reg:squarederror",
    "num_round": "50",
    "verbosity": "2",
}

instance_type = "ml.m5.4xlarge"
output_path = "s3://{}/{}/{}/output".format(bucket, prefix, "abalone-xgb")
content_type = "libsvm"

In [6]:
print(output_path)

s3://sagemaker-us-east-1-381492029181/sagemaker/DEMO-xgboost-spot/abalone-xgb/output


If Spot instances are used, the training job can be interrupted, causing it to take longer to start or finish. If a training job is interrupted, a checkpointed snapshot can be used to resume from a previously saved point and can save training time (and cost).

To enable checkpointing for Managed Spot Training using SageMaker XGBoost we need to configure three things:

1. Enable the **train_use_spot_instances** constructor arg - a simple self-explanatory boolean.

2. Set the **train_max_wait constructor** arg - this is an int arg representing the amount of time you are willing to wait for Spot infrastructure to become available. Some instance types are harder to get at Spot prices and you may have to wait longer. You are not charged for time spent waiting for Spot infrastructure to become available, you’re only charged for actual compute time spent once Spot instances have been successfully procured.

3. Setup a **checkpoint_s3_uri** constructor arg - this arg will tell SageMaker an S3 location where to save checkpoints. While not strictly necessary, checkpointing is highly recommended for Manage Spot Training jobs due to the fact that Spot instances can be interrupted with short notice and using checkpoints to resume from the last interruption ensures you don’t lose any progress made before the interruption.

Feel free to toggle the **train_use_spot_instances** variable to see the effect of running the same job using regular (a.k.a. “On Demand”) infrastructure.

Note that **train_max_wait** can be set if and only if **train_use_spot_instances** is enabled and must be greater than or equal to **train_max_run**.

In [7]:
import time
from sagemaker.inputs import TrainingInput

job_name = "DEMO-xgboost-spot-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print("Training job", job_name)

use_spot_instances = True
max_run = 3600
max_wait = 7200 if use_spot_instances else None
checkpoint_s3_uri = (
    "s3://{}/{}/checkpoints/{}".format(bucket, prefix, job_name) if use_spot_instances else None
)
print("Checkpoint path:", checkpoint_s3_uri)

estimator = sagemaker.estimator.Estimator(
    container,
    role,
    hyperparameters=hyperparameters,
    instance_count=1,
    instance_type=instance_type,
    volume_size=5,  # 5 GB
    output_path=output_path,
    sagemaker_session=sagemaker.Session(),
    use_spot_instances=use_spot_instances,
    max_run=max_run,
    max_wait=max_wait,
    checkpoint_s3_uri=checkpoint_s3_uri,
)

in_data = "s3://{}/{}/{}".format(bucket, prefix, "train")
print("in data:", in_data)

train_input = TrainingInput(
    s3_data=in_data, content_type="libsvm"
)
estimator.fit({"train": train_input}, job_name=job_name)

Training job DEMO-xgboost-spot-2024-09-12-04-18-15
Checkpoint path: s3://sagemaker-us-east-1-381492029181/sagemaker/DEMO-xgboost-spot/checkpoints/DEMO-xgboost-spot-2024-09-12-04-18-15


INFO:sagemaker:Creating training-job with name: DEMO-xgboost-spot-2024-09-12-04-18-15


in data: s3://sagemaker-us-east-1-381492029181/sagemaker/DEMO-xgboost-spot/train
2024-09-12 04:18:17 Starting - Starting the training job...
2024-09-12 04:18:30 Starting - Preparing the instances for training...
2024-09-12 04:19:08 Downloading - Downloading the training image......
2024-09-12 04:20:14 Training - Training image download completed. Training in progress.
2024-09-12 04:20:14 Uploading - Uploading generated training model[2024-09-12 04:20:10.304 ip-10-2-203-119.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-09-12 04:20:10.325 ip-10-2-203-119.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-09-12:04:20:10:INFO] Imported framework sagemaker_xgboost_container.training
[2024-09-12:04:20:10:INFO] Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
[2024-09-12:04:20:10:INFO] No GPUs detected (normal if no gpus installed)
[2024-09-12:04:20:10:INFO] Running XGBoost Sagema


2024-09-12 04:20:27 Completed - Training job completed
Training seconds: 95
Billable seconds: 32
Managed Spot Training savings: 66.3%


## Savings
Towards the end of the job you should see two lines of output printed:

* **Training seconds: X** : This is the actual compute-time your training job spent
* **Billable seconds: Y** : This is the time you will be billed for after Spot discounting is applied.

If you enabled the **train_use_spot_instances**, then you should see a notable difference between X and Y signifying the cost savings you will get for having chosen Managed Spot Training. This should be reflected in an additional line: - **Managed Spot Training savings: (1-Y/X)*100 %**

### Train with Automatic Model Tuning (HPO) and Spot Training enabled
You could also train with Amazon SageMaker Automatic Model Tuning. AMT, also known as hyperparameter tuning, finds the best version of a model by running many training jobs on your dataset using the algorithm and ranges of hyperparameters that you specify. It then chooses the hyperparameter values that result in a model that performs the best, as measured by a metric that you choose. We will use a HyperparameterTuner object to interact with Amazon SageMaker hyperparameter tuning APIs.

The code sample below shows you how to use the HyperParameterTuner and Spot Training together.

In [11]:
from sagemaker.tuner import ContinuousParameter, IntegerParameter
from sagemaker.utils import name_from_base
from sagemaker.tuner import HyperparameterTuner


# You can select from the hyperparameters supported by the model, and configure ranges of values to be searched for training the optimal model.(https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-define-ranges.html)
hyperparameter_ranges = {
    "max_depth": IntegerParameter(0, 10, scaling_type="Auto"),
    "num_round": IntegerParameter(1, 4000, scaling_type="Auto"),
    "alpha": ContinuousParameter(0, 2, scaling_type="Auto"),
    "subsample": ContinuousParameter(0.5, 1, scaling_type="Auto"),
    "min_child_weight": ContinuousParameter(0, 120, scaling_type="Auto"),
    "gamma": ContinuousParameter(0, 5, scaling_type="Auto"),
    "eta": ContinuousParameter(0.1, 0.5, scaling_type="Auto"),
}

# Increase the total number of training jobs run by AMT, for increased accuracy (and training time).
max_jobs = 6
# Change parallel training jobs run by AMT to reduce total training time, constrained by your account limits.
# if max_jobs=max_parallel_jobs then Bayesian search turns to Random.
max_parallel_jobs = 2

hp_tuner = HyperparameterTuner(
    estimator,
    "validation:rmse",
    hyperparameter_ranges,
    max_jobs=max_jobs,
    max_parallel_jobs=max_parallel_jobs,
    objective_type="Minimize",
    base_tuning_job_name=job_name,
)

# Launch a SageMaker Tuning job to search for the best hyperparameters
# In this case, the tuner requires a `validation` channel to emit the validation:rmse metric.
# Since we only created a `train` channel, we re-use it for validation.
hp_tuner.fit({"train": train_input, "validation": train_input})

INFO:sagemaker:Creating hyperparameter tuning job with name: DEMO-xgboost-spot-20-240912-0449


............................................................................................!


### Enabling checkpointing for script mode
An additional mode of operation is to run customizable scripts as part of the training and inference jobs. See [this notebook](https://sagemaker-examples.readthedocs.io/en/latest/introduction_to_amazon_algorithms/xgboost_abalone/xgboost_abalone_dist_script_mode.html) for details on how to setup script mode.

Here we highlight the specific changes that would enable checkpointing and use Spot instances.

Checkpointing in the framework mode for SageMaker XGBoost can be performed using two convenient functions:

* `save_checkpoint`:** this returns a callback function that performs checkpointing of the model for each round. This is passed to XGBoost as part of the `callbacks` <https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.train>`  argument.
* `load_checkpoint`: This is used to load existing checkpoints to ensure training resumes from where it previously stopped.

Both functions take the checkpoint directory as input, which in the below example is set to `/opt/ml/checkpoints`. The primary arguments that change for the `xgb.train` call are

1. `xgb_model`: This refers to the previous checkpoint (saved from a previously run partial job) obtained by `load_checkpoint`. This would be None if no previous checkpoint is available.
2. `callbacks`: This contains a function that performs the checkpointing

Updated script looks like the following.

In [12]:
CHECKPOINTS_DIR = '/opt/ml/checkpoints'   # default location for Checkpoints
callbacks = [save_checkpoint(CHECKPOINTS_DIR)]
prev_checkpoint, n_iterations_prev_run = load_checkpoint(CHECKPOINTS_DIR)
bst = xgb.train(
        params=train_hp,
        dtrain=dtrain,
        evals=watchlist,
        num_boost_round=(args.num_round - n_iterations_prev_run),
        xgb_model=prev_checkpoint,
        callbacks=callbacks
    )

NameError: name 'save_checkpoint' is not defined

### Using the SageMaker XGBoost Estimator
The XGBoost estimator class in the SageMaker Python SDK allows us to run that script as a training job on the Amazon SageMaker managed training infrastructure. We’ll also pass the estimator our IAM role, the type of instance we want to use, and a dictionary of the hyperparameters that we want to pass to our script.

In [13]:
from sagemaker.xgboost.estimator import XGBoost

job_name = "DEMO-xgboost-regression-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print("Training job", job_name)
checkpoint_s3_uri = (
    "s3://{}/{}/checkpoints/{}".format(bucket, prefix, job_name) if use_spot_instances else None
)
print("Checkpoint path:", checkpoint_s3_uri)

xgb_script_mode_estimator = XGBoost(
    entry_point="abalone.py",
    hyperparameters=hyperparameters,
    role=role,
    instance_count=1,
    instance_type=instance_type,
    framework_version="1.7-1",
    output_path="s3://{}/{}/{}/output".format(bucket, prefix, "xgboost-script-mode"),
    use_spot_instances=use_spot_instances,
    max_run=max_run,
    max_wait=max_wait,
    checkpoint_s3_uri=checkpoint_s3_uri,
)

INFO:sagemaker.image_uris:Ignoring unnecessary Python version: py3.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.m5.4xlarge.


Training job DEMO-xgboost-regression-2024-09-12-06-39-41
Checkpoint path: s3://sagemaker-us-east-1-381492029181/sagemaker/DEMO-xgboost-spot/checkpoints/DEMO-xgboost-regression-2024-09-12-06-39-41


Training is as simple as calling **fit** on the Estimator. This will start a SageMaker Training job that will download the data, invoke the entry point code (in the provided script file), and save any model artifacts that the script creates. In this case, the script requires a **train** and a **validation** channel. Since we only created a **train** channel, we re-use it for validation.

In [14]:
xgb_script_mode_estimator.fit({"train": train_input, "validation": train_input}, job_name=job_name)

FileNotFoundError: [Errno 2] No such file or directory: 'abalone.py'

As previously stated, the estimator can also be passed to the HyperparameterTuner object to interact with the Amazon SageMaker hyperparameter tuning APIs and create a HyperParameter Tuning Job. Hyper Parameters are automatically tuned which in most cases results in a more accurate model.

In [ ]:
hp_tuner = HyperparameterTuner(
    xgb_script_mode_estimator,
    "validation:rmse",
    hyperparameter_ranges,
    max_jobs=max_jobs,
    max_parallel_jobs=max_parallel_jobs,
    objective_type="Minimize",
    base_tuning_job_name=job_name,
)

# Launch a SageMaker Tuning job to search for the best hyperparameters
# In this case, the tuner requires a `validation` channel to emit the validation:rmse metric.
# Since we only created a `train` channel, we re-use it for validation.
hp_tuner.fit({"train": train_input, "validation": train_input})